In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm

Approach taken from NCSS' guide [Two Proportions – Non-Inferiority, Superiority, Equivalence, and Two-Sided Tests vs. a Margin](https://ncss-wpengine.netdna-ssl.com/wp-content/themes/ncss/pdf/Procedures/NCSS/Two_Proportions-Non-Inferiority,_Superiority,_Equivalence,_and_Two-Sided_Tests_vs_a_Margin.pdf), as with previous statistical approaches (i.e. as their guides include detailed formulae).

In [45]:
data = {'test': [60, 1000-60], 'ctrl': [70, 1000-70]}
# N.B. convention = test is n1 and p1, ctrl is n2 and p2, hence the order

df = pd.DataFrame.from_dict(data, orient='index',
                            columns=['don', 'non'])
print(df)

      don  non
test   60  940
ctrl   70  930


In [46]:
delta = -0.01  # The non-inferiority margin

Take the test statistic from p19-20.

In [47]:
df['total'] = df.sum(axis=1)
[n1, n2] = df['total']
N = n1 + n2
[p1, p2] = df['don'] / df['total']
print('Test: {}, {}\nCtrl: {}, {}'.format(n1, p1, n2, p2))

Test: 1000, 0.06
Ctrl: 1000, 0.07


In [48]:
l0 = df.loc['ctrl', 'don'] * delta * (1 - delta)
l1 = (n2*delta - N - 2*df.loc['ctrl', 'don']) * delta + df['don'].sum()
l2 = (N + n2)*delta - N - df['don'].sum()
l3 = N
for i in [l0, l1, l2, l3]:
    print(i)

-0.7070000000000001
151.5
-2160.0
2000


In [49]:
C = (l2**3 / (27 * (l3**3))) - (l1*l2)/(6 * (l3**2)) + l0/(2*l3)
B = np.sign(C) * np.sqrt((l2**2)/(9 * (l3**2)) - l1/(3*l3))
A = (1/3) * (np.pi + np.arccos(C/(B**3)))
print(C, B, A)

-0.033197750000000005 -0.3230325061042619 1.10529455981647


In [50]:
p2_cd = 2*B*np.cos(A) - l2/(3*l3)  # constrained p
p1_cd = p2_cd + delta
print(p1_cd, p2_cd, p1_cd - p2_cd)

0.059999999999999505 0.06999999999999951 -0.010000000000000002


In [51]:
z_MND_num = p1 - p2 - delta
z_MND_den = (p1_cd*(1-p1_cd)/n1 + p2_cd*(1-p2_cd)/n2) * N / (N-1)
z_MND_den = np.sqrt(z_MND_den)
z_MND = z_MND_num / z_MND_den
print(z_MND)

-7.86689802094413e-16


p38 => this is treated as a standard z-score, and therefore is turned into a p-value by:

In [52]:
norm.cdf(z_MND)

0.49999999999999967

It being p=0.5 for a situ where the observed difference seen is the actual NIM makes sense! Looks good => to functionify it:

In [68]:
def mndifference(datafr, nim):
    """Perform the Miettinen-Nurminen Large-Sample Score Test of the Difference.
    
    Parameters
    ----------
    datafr : pd.DataFrame
        The data to run the test on.
        Columns: don, non, total
        Index: test, ctrl
        N.B. the convention here is that the experiment is indexed as 1, and the control as 2, in reverse of normal.
    nim : float, <0 when lower is worse.
        The Non-Inferiority Margin: the amount that P1 can be less than P2 and you still conclude that
        group 1 (treatment) is not inferior to group 2 (control).

    Returns
    -------
    ? : ?
        Blah
    """
    [n1, n2] = datafr['total']
    N = n1 + n2
    [p1, p2] = datafr['don'] / datafr['total']
    
    l0 = datafr.loc['ctrl', 'don'] * nim * (1 - nim)
    l1 = (n2*nim - N - 2*datafr.loc['ctrl', 'don']) * nim + datafr['don'].sum()
    l2 = (N + n2)*nim - N - datafr['don'].sum()
    l3 = N
    
    C = (l2**3 / (27 * (l3**3))) - (l1*l2)/(6 * (l3**2)) + l0/(2*l3)
    B = np.sign(C) * np.sqrt((l2**2)/(9 * (l3**2)) - l1/(3*l3))
    A = (1/3) * (np.pi + np.arccos(C/(B**3)))
    
    p2_cd = 2*B*np.cos(A) - l2/(3*l3)  # constrained p
    p1_cd = p2_cd + nim
    
    z_MND_num = p1 - p2 - nim
    z_MND_den = (p1_cd*(1-p1_cd)/n1 + p2_cd*(1-p2_cd)/n2) * N / (N-1)
    z_MND_den = np.sqrt(z_MND_den)
    z_MND = z_MND_num / z_MND_den

    return z_MND, norm.cdf(1-z_MND)


print(mndifference(df, delta))

(-7.86689802094413e-16, 0.8413447460685431)


In [92]:
diff_range = 300
differences = range(-diff_range, diff_range+1, 100)
# step_nim = -0.04

for diff in differences:
    ctrl_convs = 1200
    cell_size = 10000
    test_convs = ctrl_convs + diff
    step_nim = - (ctrl_convs / cell_size) / 3
    step_data = {'test': [test_convs, cell_size-test_convs, cell_size],
                 'ctrl': [ctrl_convs, cell_size-ctrl_convs, cell_size]}
    step_df = pd.DataFrame.from_dict(step_data, orient='index', columns=['don', 'non', 'total'])
    step_z, step_p = mndifference(step_df, step_nim)
    step_p1 = step_df.loc['test', 'don'] / step_df.loc['test', 'total']
    step_p2 = step_df.loc['ctrl', 'don'] / step_df.loc['ctrl', 'total']
    print(step_df, '\nctr1 = {}\nctr2 = {}\nz: {}\np: {}\n'.format(step_p1, step_p2, step_z, step_p))

       don   non  total
test   900  9100  10000
ctrl  1200  8800  10000 
ctr1 = 0.09
ctr2 = 0.12
z: 2.303390756582445
p: 0.0962206953059152

       don   non  total
test  1000  9000  10000
ctrl  1200  8800  10000 
ctr1 = 0.1
ctr2 = 0.12
z: 4.500807405945609
p: 0.00023192546456305246

       don   non  total
test  1100  8900  10000
ctrl  1200  8800  10000 
ctr1 = 0.11
ctr2 = 0.12
z: 6.606882485347294
p: 1.0300171368606556e-08

       don   non  total
test  1200  8800  10000
ctrl  1200  8800  10000 
ctr1 = 0.12
ctr2 = 0.12
z: 8.633283165227821
p: 1.1442473257896606e-14

       don   non  total
test  1300  8700  10000
ctrl  1200  8800  10000 
ctr1 = 0.13
ctr2 = 0.12
z: 10.589502001723977
p: 4.425559304295249e-22

       don   non  total
test  1400  8600  10000
ctrl  1200  8800  10000 
ctr1 = 0.14
ctr2 = 0.12
z: 12.483393493598182
p: 7.993987341415403e-31

       don   non  total
test  1500  8500  10000
ctrl  1200  8800  10000 
ctr1 = 0.15
ctr2 = 0.12
z: 14.321550228593223
p: 8.67338633867

p38 => p<0.05 means that the experimental treatment is non-inferior to the standard treatment. So the p-value getting smaller as the test cvr increases is correct, as smaller p => more sure the test is non-inferior

But it looks like bigger sample sizes leads to an equivalent difference (e.g. of -0.02) getting a smaller p-value -- shouldn't it be the reverse? We should be more certain that a test is inferior if the sample size increases i.e. a sample size increase should lead 

# Sample size calculation

From [Power calculator for binary outcome non-inferiority trial](https://www.sealedenvelope.com/power/binary-noninferior/) (Sealed Envelope Ltd, 2012, accessed Fri Jul 17 2020):

> n = f(α, β) × [πs × (100 − πs) + πe × (100 − πe)] / (πs − πe − d)2
where πs and πe are the true percent 'success' in the standard and experimental treatment group respectively, and
> 
> f(α, β) = [Φ-1(α) + Φ-1(β)]2
Φ-1 is the cumulative distribution function of a standardised normal deviate.

Which you turn into the power by (cf. [Calculate Sample Size Needed to Compare 2 Proportions: 2-Sample Non-Inferiority or Superiority](http://powerandsamplesize.com/Calculators/Compare-2-Proportions/2-Sample-Non-Inferiority-or-Superiority):

\begin{equation*}
n_A=\kappa n_B \;\text{ and }\;
		n_B=\left(\frac{p_A(1-p_A)}{\kappa}+p_B(1-p_B)\right)
			\left(\frac{z_{1-\alpha}+z_{1-\beta}}{p_A-p_B-\delta}\right)^2
\end{equation*}
\begin{equation*}
1-\beta=
			\Phi\left(z-z_{1-\alpha/2}\right)+\Phi\left(-z-z_{1-\alpha/2}\right)
			\quad ,\quad z=\frac{p_A-p_B-\delta}{\sqrt{\frac{p_A(1-p_A)}{n_A}+\frac{p_B(1-p_B)}{n_B}}}
\end{equation*}

N.B. http://albertotb.com/Equivalence-between-distribution-functions-in-R-and-Python/ => that pnorm is cdf and qnorm is inverse cdf i.e. ppf.

In [75]:
p_standard = 0.10
p_exp = 0.10
d = -0.20
alpha = 0.05
beta = 0.20

def noninferior_sample_size(p_ctrl, p_test, margin, alpha=0.05, beta=0.2):
    """Perform a sample size calculation for a non-inferiority / superiority test, based around a given margin."""
    f = (norm.ppf(alpha) + norm.ppf(beta)) ** 2
    n = f * (p_ctrl*(1-p_ctrl) + p_test*(1-p_test)) / ((p_ctrl - p_test - margin) ** 2)
    # z = (p_standard - p_exp - d) / np.sqrt(p_standard*(1-p_standard)/n + p_exp*(1-p_exp)/n)
    # power = norm.cdf(z-norm.ppf(1-alpha)) + norm.cdf(-z-norm.ppf(1-alpha))
    # Not necessary - as we're specifying a given power
    return int(np.ceil(n))

for percent_exp in range(7, 13+1):
    p_exp = percent_exp / 100
    print(p_standard, p_exp, d, noninferior_sample_size(p_standard, p_exp, d))

0.1 0.07 -0.2 19
0.1 0.08 -0.2 21
0.1 0.09 -0.2 25
0.1 0.1 -0.2 28
0.1 0.11 -0.2 33
0.1 0.12 -0.2 38
0.1 0.13 -0.2 44


In [76]:
p_range = [i/100 for i in range(8, 12+1)]
margin_range = [i/100 for i in range(5, 40+1, 5)]
sample_sizes = pd.DataFrame(index=margin_range, columns=p_range)
for expected_p in p_range:
    for expected_margin in margin_range:
        sample_sizes.loc[expected_margin, expected_p] = noninferior_sample_size(p_standard, expected_p, -expected_margin)
print(sample_sizes)

      0.08  0.09  0.10  0.11  0.12
0.05   207   296   446   727  1344
0.10    71    88   112   144   189
0.15    35    42    50    60    72
0.20    21    25    28    33    38
0.25    14    16    18    21    23
0.30    10    12    13    14    16
0.35     8     9    10    11    12
0.40     6     7     7     8     9


This is implying that the better a test is, the more data you'd need?? No!